## 1997 Processing

In [1]:
import pandas as pd

In [2]:
#read in the 1997 data
data_1997 = pd.read_csv('1997.csv')

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data_1997.head()

,Unnamed: 0,GEO_TTL,NAICS1997_TTL,EMP,ESTAB,OPTAX,NAICS1997,state,place
0,0,"Arab, AL",Manufacturing,2661,26,99,31-33,1,2116
1,1,"Birmingham, AL",Manufacturing,19057,382,99,31-33,1,7000
2,2,"Boaz, AL",Manufacturing,4488,20,99,31-33,1,7912
3,3,"Decatur, AL",Manufacturing,8495,124,99,31-33,1,20104
4,4,"Dothan, AL",Manufacturing,8269,102,99,31-33,1,21184


In [ ]:
#Collect the state names and abbreviation data
!curl https://raw.githubusercontent.com/fitnr/addfips/master/addfips/data/states.csv?accessType = Download > states.csv

In [5]:
#turn the states.csv into a dataframe
states = pd.read_csv('states.csv')
states.head()

,name,postal,fips
0,Alabama,AL,1
1,Alaska,AK,2
2,Arizona,AZ,4
3,Arkansas,AR,5
4,California,CA,6


In [6]:
#get the abbreviation and the proper FIPs codes for joining
states['fips'] = states['fips'].astype(str)
data_1997['state'] = data_1997['state'].astype(str)

states['fips'] = states.fips.str.zfill(2)
data_1997['state'] = data_1997.state.str.zfill(2)

states_cleaned = states[['postal','fips']]

states_cleaned.head()

,postal,fips
0,AL,01
1,AK,02
2,AZ,04
3,AR,05
4,CA,06


In [7]:
#join the states and 1997 data
data_1997_us = data_1997.merge(states_cleaned, left_on = 'state', right_on = 'fips', how = 'inner')

In [8]:
#remove territories and county-balances
not_us_states = ['60','66','69','72','78']

data_1997_states = data_1997_us.loc[~data_1997_us['fips'].isin(not_us_states)]
data_1997_cities = data_1997_states.loc[~data_1997_states['GEO_TTL'].str.contains('Balance',na=False)]

In [9]:
#Remove duplicate rows for Tax codes 99 and A
tax_codes = ['99','A']
data_1997_cities_all_occupation_1 = data_1997_cities.loc[data_1997_cities['OPTAX'].isin(tax_codes)]
data_1997_cities_remove_dupes = data_1997_cities_all_occupation_1.drop_duplicates()
data_1997_cities_remove_dupes.head()

,Unnamed: 0,GEO_TTL,NAICS1997_TTL,EMP,ESTAB,OPTAX,NAICS1997,state,place,postal,fips
13060,655455,"Anniston, AL",Real estate,121,31,99,531,01,1852,AL,01
13061,655456,"Jacksonville, AL",Real estate,0,5,99,531,01,38272,AL,01
13063,655458,"Oxford, AL (Calhoun County Part)",Real estate,0,5,99,531,01,57576,AL,01
13064,655459,"Piedmont, AL (Calhoun County Part)",Real estate,0,1,99,531,01,59640,AL,01
13065,655691,"Oneonta, AL",Real estate,0,4,99,531,01,57000,AL,01


In [10]:
#Begin feature engineering a unique city_state code by splitting GEO_TTL by comma
data_1997_cities_remove_dupes['city_name'] = data_1997_cities_remove_dupes.GEO_TTL.str.split(',')

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [11]:
#create a state name column. Will have extra elements than desired
data_1997_cities_remove_dupes['state_name_unclean'] = data_1997_cities_remove_dupes.city_name.str[1]

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [12]:
#Create a city name column
data_1997_cities_remove_dupes['city_name'] = data_1997_cities_remove_dupes.city_name.str[0]

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [13]:
#Clean the state name column to just the abbreviated state name
data_1997_cities_remove_dupes['state_name_cleaned'] = data_1997_cities_remove_dupes.state_name_unclean.str[:3]

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [14]:
#create the unique city_state code
data_1997_cities_remove_dupes['city_state_code'] = data_1997_cities_remove_dupes['city_name'] + '_' + data_1997_cities_remove_dupes['state_name_cleaned']

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [16]:
#create the list of creative professionals within the NAICS titles
creative_class = ['Heavy and civil engineering construction',
'Veneer, plywood, and engineered wood product manufacturing',
'Pharmaceutical and medicine manufacturing',
'Electronic computer manufacturing',
'Computer terminal and other computer peripheral equipment manufacturing',
'Navigational, measuring, electromedical, and control instruments manufacturing',
'Electromedical and electrotherapeutic apparatus manufacturing',
'Surgical and medical instrument manufacturing',
'Computer and computer peripheral equipment and software merchant wholesalers',
'Computer and computer peripheral equipment merchant wholesalers',
'Surgical, medical, and hospital supplies merchant wholesalers',
'Direct life, health, and medical insurance carriers',
'Direct health and medical insurance carriers',
'Direct insurance (except life, health, and medical) carriers',
'Offices of lawyers',
'Legal aid societies and similar legal services',
'Other legal services',
'All other legal services',
'Accounting, tax preparation, bookkeeping, and payroll services',
'Other accounting services',
'Architectural, engineering, and related services',
'Custom computer programming services',
'Other computer related services',
'Scientific research and development services',
'Research and development in the physical, engineering, and life sciences',
'Research and development in the physical, engineering, and life sciences',
'Research and development in biotechnology',
'Research and development in the physical, engineering, and life sciences (except biotechnology)',
'Research and development in the physical and engineering sciences',
'Research and development in other life sciences',
'Research and development in the social sciences and humanities',
'Marketing research and public opinion polling',
'Economic or industrial planning or development organization',
'Business schools and computer and management training',
'Professional and management development training',
'Art, drama, and music schools',
'Family planning centers',
'HMO medical centers',
'Freestanding ambulatory surgical and emergency centers',
'General medical and surgical hospitals',
'General medical and surgical hospitals, government',
'General medical and surgical hospitals, except government',
'Residential intellectual and developmental disability, mental health, and substance abuse facilities',
'Residential intellectual and developmental disability facilities',
'Musical groups and artists',
'Symphony orchestras and chamber music organizations',
'Other music groups and artists',
'Agents and managers for artists, athletes, entertainers, and other public figures',
'Independent artists, writers, and performers']

In [17]:
#case when arguement to define a profession as creative or not
data_1997_cities_remove_dupes['class'] = pd.np.where(
     data_1997_cities_remove_dupes['NAICS1997_TTL'].isin(creative_class), 
    '1_creative',  
    pd.np.where(~data_1997_cities_remove_dupes['NAICS1997_TTL'].isin(creative_class), 
    '2_regular','3_none'
     ))

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [18]:
#reset the index
data_1997 = data_1997_cities_remove_dupes.reset_index()

In [19]:
#select columns of interest
data_1997 = data_1997[['city_state_code','EMP','ESTAB','class']]

In [20]:
#groupby the data and unstack it
econ_data_1997 = data_1997.groupby(['city_state_code','class']).sum().unstack()

In [21]:
#reset the index and rename the columns
econ_data_final = econ_data_1997.reset_index()
econ_data_final.columns = ['city_state','creative_employees','regular_employees',
                           'creative_establishments','regular_establishments']

In [23]:
#remove any extra characters and lower case the texts from the city_state code
econ_data_final['city_state'] = econ_data_final['city_state'].str.replace(' ','')
econ_data_final['city_state'] = econ_data_final['city_state'].str.replace('.','')
econ_data_final['city_state'] = econ_data_final['city_state'].str.replace(',','')
econ_data_final['city_state'] = econ_data_final['city_state'].str.lower()

In [24]:
#write out the final csv
econ_data_final.to_csv('1997_creative_class.csv')